On local runtime bc RAM is better

DLC version: 2.3.5 is ok

need to copy over:
- video folder analyzed pickles
  - must go into video folder and select all and then download zip. Otherwise will miss files.
  - will give multiple zips probably
  - double check that you have the correct number of files downloaded
- dlc models
- config.yaml

#troubleshooting

In [ ]:
pip install deeplabcut[gui]

#install other packages and setup

In [ ]:
import os
import deeplabcut
import numpy as np
import pandas as pd
import pickle
from deeplabcut.utils import auxfun_multianimal, auxiliaryfunctions
import random
from pathlib import Path
import glob

ModuleNotFoundError: ignored

# which model

In [ ]:
project_path = "D:\\paulazhu\\demo-me-2021-07-14\\"
config_path = os.path.join(project_path, "config.yaml")
tracklets_config_path = os.path.join(project_path, "tracklets_config.yaml")

In [ ]:
SHUFFLE = 1
TRACK_METHOD = "ellipse"  # Could also be "box", but "ellipse" was found to be more robust on this dataset.

In [ ]:
from deeplabcut.utils.auxiliaryfunctions import edit_config
SNAPSHOT = 4 # not the actual number, but its index within the folder starting from 0
edit_config(config_path, {'snapshotindex': SNAPSHOT})

{'Task': 'demo', 'scorer': 'me', 'date': 'Jul14', 'multianimalproject': True, 'identity': None, 'project_path': '/content/drive/MyDrive/training/demo-me-2021-07-14', 'video_sets': {'D:\\paulazhu\\demo-me-2021-07-14\\videos\\221002_PZ71_1.avi': {'crop': '0, 287, 0, 166'}, 'D:\\paulazhu\\demo-me-2021-07-14\\videos\\221009_PZ70_1.avi': {'crop': '0, 286, 0, 169'}, 'D:\\paulazhu\\demo-me-2021-07-14\\videos\\221009_PZ71_1.avi': {'crop': '0, 289, 0, 168'}, 'D:\\paulazhu\\demo-me-2021-07-14\\videos\\221016_PZ70_1.avi': {'crop': '0, 293, 0, 172'}, 'D:\\paulazhu\\demo-me-2021-07-14\\videos\\221016_PZ71_1.avi': {'crop': '0, 292, 0, 171'}, 'D:\\paulazhu\\demo-me-2021-07-14\\videos\\221024_PZ70_1.avi': {'crop': '0, 288, 0, 169'}, 'D:\\paulazhu\\demo-me-2021-07-14\\videos\\221024_PZ71_1.avi': {'crop': '0, 291, 0, 173'}, 'D:\\paulazhu\\demo-me-2021-07-14\\videos\\221106_PZ70_1.avi': {'crop': '0, 285, 0, 168'}, 'D:\\paulazhu\\demo-me-2021-07-14\\videos\\221106_PZ71_1.avi': {'crop': '0, 287, 0, 168'}, 

# Convert_detections2tracklets on local runtime

In [ ]:
#for video in [project_path + "videos\\video.avi"]:
for video in glob.glob(project_path + "/videos/" + "*.avi"):
  deeplabcut.convert_detections2tracklets(
      config_path,
      [video],
      videotype='avi',
      shuffle=SHUFFLE,
      track_method=TRACK_METHOD,
      ignore_bodyparts=["tail1", "tail2", "tailend"],  # Some body parts can optionally be ignored during tracking for better assembly (but they are used later)
  )


Using snapshot-50000 for model /content/drive/MyDrive/training/demo-me-2021-07-14/dlc-models/iteration-2/demoJul14-trainset95shuffle1


/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Activating extracting of PAFs
Starting to analyze %  /content/drive/MyDrive/training/demo-me-2021-07-14/videos/221002_PZ71_1.avi
Video already analyzed! /content/drive/MyDrive/training/demo-me-2021-07-14/videos/221002_PZ71_1DLC_resnet50_demoJul14shuffle1_50000.h5
The videos are analyzed. Time to assemble animals and track 'em... 
 Call 'create_video_with_all_detections' to check multi-animal detection quality before tracking.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Using snapshot-50000 for model /content/drive/MyDrive/training/demo-me-2021-07-14/dlc-models/iteration-2/demoJul14-trainset95shuffle1
Processing...  /content/drive/MyDrive/training/demo-me-2021-07-14/videos/221002_PZ71_1.avi
Analyzing /content/drive/MyDrive/training/demo-me-2021-07-14/videos/221002_PZ71_1DLC_resnet50_demoJul14shuffle1_50000.h5


  7%|▋         | 2702/36164 [00:42<08:42, 64.09it/s]


KeyboardInterrupt: ignored

# Stitch and filter tracklets

In [ ]:
deeplabcut.stitch_tracklets(
  config_path,
  [video],
  videotype='avi',
  shuffle=SHUFFLE,
  track_method=TRACK_METHOD,
  n_tracks=2, # if 2 doesn't work, try 3 or 1 depending on how clear 2 mice is; also seems to mess up create_labeled_video when not 3
)
#Filter the predictions to remove small jitter, if desired:
deeplabcut.filterpredictions(config_path,
                              [video],
                              shuffle=SHUFFLE,
                              videotype='avi',
                              track_method = TRACK_METHOD)


# Make videos

In [ ]:
from deeplabcut.utils.auxiliaryfunctions import edit_config
PCUTOFF = 0.1 # previously 0.01
edit_config(config_path, {'pcutoff': PCUTOFF})

# might not give any labels when n_tracks not 3 like in config.yaml
deeplabcut.create_labeled_video(
  config_path,
  [video],
  videotype='avi',
  shuffle=SHUFFLE,
  color_by="individual",
  keypoints_only=False,
  draw_skeleton=True,
  filtered=False,
  track_method=TRACK_METHOD,
)

# works better i think because different n_tracks
deeplabcut.create_video_with_all_detections(
  config_path,
  [video],
  videotype='avi',
  shuffle=SHUFFLE,
  )